# COMP 5212 Programming Assignment 1
## Image classification with Logistic Regression and Support Vector Machine

### Load data
* Load minist data and store the data to $home$ directory
* create dataloaders for training and testing

In [10]:
%load_ext autoreload
%autoreload 2

In [1]:
#Don't change batch size
batch_size = 64

from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

In [2]:
train_data = datasets.MNIST('~/data/mnist', train=True, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ]))
test_data = datasets.MNIST('~/data/mnist', train=False, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))
subset_indices = ((train_data.train_labels == 0) +
                  (train_data.train_labels == 1)).nonzero().squeeze(1)
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=batch_size, shuffle=False, sampler=SubsetRandomSampler(subset_indices))


subset_indices = ((test_data.test_labels == 0) +
                  (test_data.test_labels == 1)).nonzero().squeeze(1)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=batch_size, shuffle=False, sampler=SubsetRandomSampler(subset_indices))


/Users/dingdongliu/anaconda3/envs/venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/Users/dingdongliu/anaconda3/envs/venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [9]:
for i, (images, labels) in enumerate(train_loader):
    images = Variable(images.view(-1, 28*28))
    #Convert labels from 0,1 to -1,1
    labels = Variable(2*(labels.float()-0.5))
    break
images.shape, labels.shape

(torch.Size([64, 784]), torch.Size([64]))

### Train model

In [14]:
from models import Logistic_Regression, logistic_loss
from torch import optim

In [35]:
# Training the Model
# Notice that newest Pytorch merge tensor and variable, so the additional Variable wrapping is no longer required.
num_epochs = 10 ## added by me. modifiable.

model = Logistic_Regression(28*28, 1)
opt = optim.SGD(model.parameters(), lr=2e-4)

print(logistic_loss(model(images), labels))

tensor(56.4049, grad_fn=<DivBackward0>)


In [36]:
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        #Convert labels from 0,1 to -1,1
        labels = Variable(2*(labels.float()-0.5))
        
        ## TODO 
        pred = model(images)
        loss = logistic_loss(pred, labels)
        

        loss.backward()
        opt.step()
        opt.zero_grad()
        total_loss += loss.detach().numpy()

    #Print your results every epoch
    print(f"Epoch {epoch}, loss = {total_loss}")


Epoch 0, loss = 8985.25479888916
Epoch 1, loss = 8868.562831878662
Epoch 2, loss = 8852.98906326294
Epoch 3, loss = 8842.556457519531
Epoch 4, loss = 8849.8862991333
Epoch 5, loss = 8841.625713348389
Epoch 6, loss = 8838.030208587646
Epoch 7, loss = 8820.801528930664
Epoch 8, loss = 8813.966201782227
Epoch 9, loss = 8811.464641571045


### Test model

In [ ]:
# Test the Model
correct = 0.
total = 0.
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    ## Put your prediction code here
    
    correct += (prediction.view(-1).long() == labels).sum()
    total += images.shape[0]
print('Accuracy of the model on the test images: %f %%' % (100 * (correct.float() / total)))